[![Ouvrir dans Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/huggingface/education-toolkit/blob/main/03_getting-started-with-transformers.ipynb)

💡 **Bienvenue!**

Nous avons assemblé une boite à outil que les professeurs du supérieur peuvent utiliser pour préparer des séances de travaux dirigés, des cours ou des devoirs. Le contenu est autonome, de manière à ce qu'il puisse être intégrer dans un cours pré-existant. Le contenu est gratuit et utilise des technologies Open Source connues (`transformers`, `gradio`, etc).

Il est aussi possible de demander à un membre de l'équipe d'Hugging Face de présenter les tutoriels dans un de vos cours via l'initiative [ML demo.cratization tour](https://www.notion.so/ML-Demo-cratization-tour-with-66847a294abd4e9785e85663f5239652) !

Vous pouvez trouver toutes les ressources que nous avons rassemblées [ici](https://huggingface2.notion.site/Education-Toolkit-7b4a9a9d65ee4a6eb16178ec2a4f3599). 

# Tutoriel : Construire et héberger des démos d'apprentissage automatique avec Gradio ⚡ et Hugging Face 🤗

**Buts de l'apprentissage :** Le but de ce tutoriel est d'apprendre comment

1. Construire une démo rapide pour votre modèle d'apprentissage automatique en Python en utilisant la bibliothèque `gradio`.
2. Héberger les démos gratuitement avec Hugging Face *Spaces*.
3. Ajoutez votre démo à l'org Hugging Face pour votre classe ou conférence. Cela comprend :
  * Une étape de configuration pour les instructeurs (ou organisateurs de conférence)
  * Des instructions de téléchargement pour les étudiants (ou les participants à la conférence)

**Durée** : 20-40  minutes

**Prérequis** : Connaissance de Python et connaissance de base de l'apprentissage automatique.


**Auteur** : [Abubakar Abid](https://twitter.com/abidlabs) (n'hésitez pas à me contacter pour toute question sur ce tutoriel) 

Toutes ces étapes peuvent être réalisées gratuitement ! Tout ce dont vous avez besoin est un navigateur Internet et un endroit où vous pouvez écrire du Python 👩💻

## Pourquoi des démos ?

**Les démos** de modèles d'apprentissage automatique constituent une partie de plus en plus importante des cours et des conférences sur l'apprentissage automatique. Les démos permettent :

* Aux développeurs de modèles de **présenter** facilement leur travail à un large public.
* D'augmenter la **reproductibilité** de la recherche en apprentissage automatique.
* Aux divers utilisateurs de **identifier et de déboguer** plus facilement les points de défaillance des modèles.


Pour un exemple rapide de ce que nous aimerions construire, consultez l'organisation [Keras sur Hugging Face](https://huggingface.co/keras-io), qui comprend une fiche descriptive et une collection de modèles et de *Spaces* construits par la communauté Keras. Tout espace peut être ouvert dans votre navigateur et vous pouvez utiliser le modèle immédiatement, comme indiqué ici : 

![](https://i.ibb.co/7y6DGjB/ezgif-5-cc52b7e590.gif)




## 1. Créer des démonstrations rapides d'apprentissage automatique en Python à l'aide de la bibliothèque Gradio

Gradio est une bibliothèque Python très pratique qui vous permet de construire des démos web en spécifiant simplement la liste des composants d'entrée et de sortie attendus par votre modèle d'apprentissage automatique. 

Qu'est-ce que j'entends par composants d'entrée et de sortie ? Gradio est livré avec un grand nombre de composants prédéfinis pour différents types de modèles d'apprentissage automatique. Voici quelques exemples :

* Pour un **classifieur d'images**, le type d'entrée attendu est une `Image` et le type de sortie est un `Label`. 
* Pour un **modèle de reconnaissance vocale**, le composant d'entrée attendu est un `Microphone` (qui permet aux utilisateurs d'enregistrer depuis le navigateur) ou un `Audio` (qui permet aux utilisateurs de glisser-déposer des fichiers audio), tandis que le type de sortie est un `Text`. 
* Pour un **modèle de réponse aux questions**, nous attendons **2 entrées** : [`Text`, `Text`], une zone de texte pour le paragraphe et une pour la question, et le type de sortie est un seul `Text` correspondant à la réponse. 

Vous voyez l'idée... (pour tous les composants supportés, [voir la documentation](https://gradio.app/docs/))

En plus des types d'entrée et de sortie, Gradio attend un troisième paramètre, qui est la fonction de prédiction elle-même. Ce paramètre peut être ***n'importe quelle* fonction Python standard** qui prend en entrée les paramètres correspondant aux composants d'entrée et retourne les valeurs correspondant aux composants de sortie.

Assez de mots. Voyons un peu de code !

In [ ]:
# Nous installons d'abord Gradio
!pip install --quiet gradio

In [ ]:
import numpy as np

def sepia(image):
    sepia_filter = np.array(
        [[0.393, 0.769, 0.189], 
         [0.349, 0.686, 0.168], 
         [0.272, 0.534, 0.131]]
    )
    sepia_img = image.dot(sepia_filter.T)
    sepia_img /= sepia_img.max()
    return sepia_img

In [ ]:
import gradio as gr

# Écrire une ligne de Python pour créer une interface graphique simple.
gr.Interface(fn=sepia, inputs="image", outputs="image").launch();

L'exécution du code ci-dessus devrait produire une interface graphique simple à l'intérieur de ce *notebook* vous permettant de taper des entrées d'exemple et de voir la sortie retournée par votre fonction. 

Remarquez que nous définissons une `Interface` en utilisant les 3 ingrédients mentionnés précédemment :
* Une fonction
* Composant(s) d'entrée
* Composant(s) de sortie

Il s'agit d'un exemple simple pour les images, mais le même principe s'applique à tout autre type de données. Par exemple, voici une interface qui génère une tonalité musicale lorsqu'on lui fournit quelques paramètres différents (le code spécifique à l'intérieur de `generate_tone()` n'est pas important dans le cadre de ce tutoriel) :

In [ ]:
import numpy as np
import gradio as gr

def generate_tone(note, octave, duration):
    sampling_rate = 48000
    a4_freq, tones_from_a4 = 440, 12 * (octave - 4) + (note - 9)
    frequency = a4_freq * 2 ** (tones_from_a4 / 12)
    audio = np.linspace(0, int(duration), int(duration) * sampling_rate)
    audio = (20000 * np.sin(audio * (2 * np.pi * frequency))).astype(np.int16)
    return sampling_rate, audio

gr.Interface(
    generate_tone,
    [
        gr.Dropdown(["C", "C#", "D", "D#", "E", "F", "F#", "G", "G#", "A", "A#", "B"], type="index"),
        gr.Slider(4, 6, step=1),
        gr.Number(value=1, label="Duration in seconds"),
    ],
    "audio",
    title="Generate a Musical Tone!"
).launch()

**Défi n° 1** : construire une démo Gradio qui prend une image et renvoie la même image *tournée à l'envers* en moins de 10 lignes de code Python.

Il y a beaucoup plus d'exemples que vous pouvez essayer sur la [page de démarrage](https://gradio.app/getting_started/) de Gradio, qui couvre des fonctionnalités supplémentaires telles que :
* L'ajout d'exemples d'entrées
* Ajout d'un _état_ (par exemple pour les chatbots)
* Partager facilement des démos en utilisant un paramètre appelé `share` (<-- c'est plutôt cool 😎).

Il est particulièrement facile de faire une démo d'un modèle `transformers` du *Hub* de Hugging Face, en utilisant la méthode spéciale `gr.Interface.load`. 

Essayons un modèle de synthèse vocale construit par Facebook :

In [ ]:
import gradio as gr

gr.Interface.load("huggingface/facebook/fastspeech2-en-ljspeech").launch();

Voici le code pour construire une démo pour [GPT-J](https://huggingface.co/EleutherAI/gpt-j-6B), un grand modèle de langage et ajouter quelques exemples d'entrées :

In [ ]:
import gradio as gr

examples = [["The Moon's orbit around Earth has"], ["There once was a pineapple"]] # L'orbite de la Lune autour de la Terre a  ,  Il était une fois un ananas

gr.Interface.load("huggingface/EleutherAI/gpt-j-6B", examples=examples).launch();

**Défi n°2** : Allez sur le [*Hub* d'Hugging Face](https://huggingface.co/models) et choisissez un modèle qui effectue une des autres tâches supportées par la bibliothèque `transformers` (autres que les deux que vous venez de voir : génération de texte ou synthèse vocale). Créez une démo Gradio pour ce modèle en utilisant `gr.Interface.load`.

## 2. Hébergez la démo (gratuitement) sur Hugging Face *Spaces*

Une fois que vous avez créé une démo Gradio, vous pouvez l'héberger de façon permanente sur *Spaces* très facilement :

Voici les étapes pour cela (montrées dans le GIF ci-dessous) :

A. Tout d'abord, créez un compte Hugging Face si vous n'en avez pas déjà un, en visitant https://huggingface.co/ et en cliquant sur "*Sign Up*".

B. Une fois que vous êtes connecté, cliquez sur votre photo de profil et ensuite cliquez sur "*New Space*" en dessous pour arriver à cette page : https://huggingface.co/new-space

C. Donnez un nom et une licence à votre *Space*. Sélectionnez "Gradio" comme *Space SDK*, puis choisissez "Public" si vous êtes d'accord pour que tout le monde accède à votre *Space* et au code sous-jacent.

D. Vous trouverez ensuite une page qui vous fournira des instructions sur la façon de télécharger vos fichiers dans le dépôt Git de ce *Space*. Vous devrez peut-être aussi ajouter un fichier `requirements.txt` pour spécifier les éventuelles dépendances du *package* Python.

E. Une fois que vous avez poussé vos fichiers, c'est tout ! *Spaces* construira automatiquement votre démo Gradio, ce qui vous permettra de la partager avec n'importe qui, n'importe où !

![GIF](https://huggingface.co/blog/assets/28_gradio-spaces/spaces-demo-finalized.gif)



Vous pouvez même intégrer votre démo Gradio sur n'importe quel site web : dans un blog, un portfolio ou même dans un *notebook*, comme je l'ai fait avec un modèle de reconnaissance de croquis ci-dessous :

In [ ]:
from IPython.display import IFrame
IFrame(src='https://hf.space/gradioiframe/abidlabs/Draw/+', width=1000, height=800)

**Challenge #3** : Téléchargez votre démo Gradio sur *Spaces* et obtenez une URL permanente pour celle-ci. Partagez l'URL permanente avec quelqu'un (un collègue, un collaborateur, un ami, un utilisateur, etc.). Quel type de retour obtenez-vous sur votre modèle d'apprentissage automatique ?

## 3. Ajoutez votre démo à l'organisation Hugging Face pour votre cours ou votre conférence

#### **Configuration** (pour les instructeurs ou les organisateurs de conférences)

A. Tout d'abord, créez un compte Hugging Face si vous n'en avez pas déjà un, en vous rendant sur le site https://huggingface.co/ et en cliquant sur "*Sign Up*".

B. Une fois que vous êtes connecté, cliquez sur votre photo de profil et ensuite cliquez sur "*New Organization*" en dessous pour arriver à cette page : https://huggingface.co/organizations/new

C. Remplissez les informations relatives à votre classe ou conférence. Nous vous recommandons de créer une organisation distincte chaque fois qu'un cours est donné (par exemple, "Stanford-CS236g-2022") et pour chaque année de conférence.

D. Votre organisation sera créée et maintenant les utilisateurs pourront demander à être ajoutés à vos organisations en visitant la page de l'organisation.

E. En option, vous pouvez modifier les paramètres en cliquant sur le bouton "*Organization settings*". En général, pour les cours et les conférences, vous voudrez naviguer dans `Settings > Members` et définir le "*Default role for new members*"  comme étant "*write*", ce qui leur permet de soumettre des *Spaces* mais pas de modifier les paramètres. 

#### Pour les étudiants ou les participants à une conférence

A. Demandez à votre instructeur / organisateur de conférence le lien vers la page Organisation si vous ne l'avez pas déjà.

B. Visitez la page de l'organisation et cliquez sur le bouton "*Request to join this org*", si vous ne faites pas encore partie de l'organisation.

C. Ensuite, une fois que vous avez été approuvé pour rejoindre l'organisation (et que vous avez construit votre Démo Gradio et l'avez téléchargé sur *Spaces*), allez simplement dans votre *Space* et allez dans `Settings > Rename or transfer this space` et ensuite sélectionnez le nom de l'organisation sous `New owner`. Cliquez sur le bouton et le *Space* sera maintenant ajouté à votre *Space* de classe ou de conférence !   